In [2]:
import chess
import torch
import numpy as np
from typing import Literal, Union, List, Tuple


class MatrixEncoder:
    def encode(self, board: chess.Board) -> np.ndarray:
        # 12 каналов для фигур
        board_state = np.zeros((12, 8, 8), dtype=np.float32)

        # 1. Кодируем состояние доски
        for square in chess.SQUARES:
            piece = board.piece_at(square)
            if piece is not None:
                # Определяем канал:
                # 0-5: пешка, конь, слон, ладья, ферзь, король
                channel = piece.piece_type - 1
                if piece.color == chess.BLACK:
                    channel += 6
                row = square // 8
                col = square % 8
                board_state[channel, row, col] = 1.0

#         # 2. Дополнительные признаки
#         if board.has_kingside_castling_rights(chess.WHITE):
#             board_state[12][7, 4] = 1.0  # Король белых на e1
#         if board.has_queenside_castling_rights(chess.WHITE):
#             board_state[12][7, 4] = 1.0  # Король белых на e1
#         if board.has_kingside_castling_rights(chess.BLACK):
#             board_state[12][7, 0] = -1.0  # Король чёрных на e8
#         if board.has_queenside_castling_rights(chess.BLACK):
#             board_state[12][7, 0] = -1.0  # Король чёрных на e8

#         if board.ep_square is not None:
#             ep_row = board.ep_square // 8
#             ep_col = board.ep_square % 8
#             board_state[13][ep_row, ep_col] = 1.0

#         if board.peek() and board.peek().promotion is None:
#             last_move = board.peek()
#             if abs(last_move.from_square - last_move.to_square) == 16:  # Ход на две клетки
#                 double_move_row = last_move.to_square // 8
#                 double_move_col = last_move.to_square % 8
#                 board_state[14][double_move_row, double_move_col] = 1.0

        return board_state

    def get_encoded_shape(self):
        return (12, 8, 8)

In [3]:
import chess
import numpy as np
import os
import pandas as pd
from tqdm import tqdm
import random

class MovesToNpyConverter:
    def __init__(
        self,
        csv_path,
        encoder,
        output_dir,
        max_games=None,
        min_moves=10,
        splits={"train": 0.7, "val": 0.15, "test": 0.15},
    ):
        self.csv_path = csv_path
        self.encoder = encoder
        self.output_dir = output_dir
        self.max_games = max_games
        self.min_moves = min_moves
        self.splits = splits
        
        # Проверяем, что сумма долей равна 1 (с учётом погрешности float)
        assert abs(sum(splits.values()) - 1.0) < 1e-6, "Сумма долей должна быть равна 1"
        
        # Создаём поддиректории для train, val и test
        self.subdirs = {}
        for split_name in splits.keys():
            dir_path = os.path.join(output_dir, split_name)
            os.makedirs(dir_path, exist_ok=True)
            self.subdirs[split_name] = dir_path
            
        # Создаём директорию для нового CSV
        os.makedirs(output_dir, exist_ok=True)
        
    def process_games(self):
        # Читаем CSV файл
        df = pd.read_csv(self.csv_path)
        game_count = 0
        split_counts = {name: 0 for name in self.splits.keys()}
        output_data = []
        
        total = self.max_games if self.max_games is not None else len(df)
        with tqdm(total=total, desc="Processing games") as pbar:
            for idx, row in df.iterrows():
                if self.max_games is not None and game_count >= self.max_games:
                    break
                
                moves_str = row['moves']
                marks = row['marks']
                moves = moves_str.split()
                
                if len(moves) >= self.min_moves:
                    board = chess.Board()
                    positions = []
                    
                    try:
                        for move in moves:
                            chess_move = board.parse_uci(move)
                            board.push(chess_move)
                            positions.append(self.encoder.encode(board))
                            
                        # Выбираем, в какую директорию сохранять
                        rand_val = random.random()
                        cumulative_prob = 0
                        for split_name, prob in self.splits.items():
                            cumulative_prob += prob
                            if rand_val <= cumulative_prob:
                                save_dir = self.subdirs[split_name]
                                split_counts[split_name] += 1
                                break
                        
                        # Сохраняем всю партию в один .npy файл
                        filename = f"game_{game_count}.npy"
                        rel_path = os.path.join(os.path.basename(save_dir), filename)
                        game_array = np.stack(positions)
                        np.save(os.path.join(save_dir, filename), game_array)
                        
                        # Добавляем запись в выходной датасет
                        output_data.append({
                            'path': rel_path,
                            'marks': marks
                        })
                        
                        game_count += 1
                    except Exception as e:
                        print(f"Ошибка обработки игры {idx}: {e}")
                        continue
                
                pbar.update(1)
                if self.max_games is not None:
                    pbar.set_postfix_str(f"Games: {game_count}/{self.max_games}")
        
        # Сохраняем новый CSV
        output_df = pd.DataFrame(output_data)
        output_csv_path = os.path.join(self.output_dir, "_info.csv")
        output_df.to_csv(output_csv_path, index=False)
        
        print(f"Обработано {game_count} партий")
        for split_name, count in split_counts.items():
            print(f"{split_name}: {count} партий ({count / game_count * 100:.1f}%)")

In [7]:
encoder = MatrixEncoder()
converter = MovesToNpyConverter(
    csv_path="/home/jupyter/datasphere/project/full_labeled.csv",
    encoder=encoder,
    output_dir="/home/jupyter/datasphere/project/transformer_dataset",
    max_games=25_000
)
converter.process_games()

Processing games: 25006it [24:11, 17.23it/s, Games: 25000/25000]                           


Обработано 25000 партий
train: 17384 партий (69.5%)
val: 3790 партий (15.2%)
test: 3826 партий (15.3%)


In [5]:
import torch
import torch.nn as nn
import os
import json

class ResidualBlock(nn.Module):
    def __init__(self, channels):
        super().__init__()
        self.conv1 = nn.Conv2d(channels, channels, kernel_size=3, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(channels)
        self.conv2 = nn.Conv2d(channels, channels, kernel_size=3, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(channels)
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        out += residual
        out = self.relu(out)
        return out

class Board2Vec(nn.Module):
    def __init__(self, hidden_dim, output_dim):
        super().__init__()
        # Входной блок с 12 каналов (6 фигур × 2 цвета + информация про рокировку, взятие на проходе и promotion)
        self.initial = nn.Sequential(
            nn.Conv2d(12, hidden_dim, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(hidden_dim),
            nn.ReLU(inplace=True)
        )

        # Резидуальные блоки
        self.block1 = ResidualBlock(hidden_dim)
        self.block2 = ResidualBlock(hidden_dim)
        self.block3 = ResidualBlock(hidden_dim)

        # Глобальный пуллинг и финальные слои
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(inplace=True),
            nn.Dropout(0.3),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(inplace=True),
            nn.Dropout(0.3),
            nn.Linear(hidden_dim, output_dim)
        )

    def forward(self, boards: torch.Tensor):
        # boards: (batch_size, 12, 8, 8)
        x = self.initial(boards)
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)

        # Конкатенация с дополнительными признаками
        x = self.fc(x)
        return x

In [6]:
# Загрузка конфигурации модели
checkpoint_dir = "checkpoints"
with open(os.path.join(checkpoint_dir, "config.json"), "r") as f:
    config = json.load(f)

# Создание модели с параметрами из конфига
model = Board2Vec(
    hidden_dim=config["HIDDEN_DIM"],
    output_dim=config["OUTPUT_DIM"]
)

# Загрузка весов модели из чекпоинта
checkpoint_path = os.path.join(checkpoint_dir, "board2vec_epoch1.pt")
model.load_state_dict(torch.load(checkpoint_path, map_location=torch.device('cpu')))
model.eval()  # переводим модель в режим оценки

# Создание случайного тензора размера (12, 8, 8)
random_tensor = torch.randn(12, 8, 8)  # можно также использовать torch.rand для значений [0, 1)

# Добавляем размер батча (1 в данном случае)
input_tensor = random_tensor.unsqueeze(0)  # теперь размер (1, 12, 8, 8)

# Пропускаем тензор через модель
with torch.no_grad():  # отключаем вычисление градиентов
    output = model(input_tensor)

print("Input tensor shape:", input_tensor.shape)
print("Output tensor shape:", output.shape)
print("Output:", output)

Input tensor shape: torch.Size([1, 12, 8, 8])
Output tensor shape: torch.Size([1, 64])
Output: tensor([[ 1.1006, -0.4905, -0.1791,  0.5547,  0.1832,  0.7637, -1.1510,  0.2183,
          0.2191, -0.8844,  0.1198, -0.1497, -0.8179, -0.6183, -0.4953, -0.1349,
         -0.0947,  0.2664, -0.2242,  0.2766,  0.1191,  0.3455,  0.2219, -0.4242,
         -0.1339,  0.1539, -0.8319, -0.3708,  0.4171,  0.8627, -0.4695, -0.1314,
         -0.4423,  0.4281,  0.2627, -0.7244, -0.7913,  0.4238,  0.5830, -0.7772,
          0.0208,  0.0298,  0.0623,  1.0745,  0.5161,  0.3190,  1.0676, -1.0908,
          0.7710,  0.1538, -0.6977, -0.3819,  0.6135, -0.9161, -1.0715, -0.0804,
          0.7776,  0.3000, -0.3934,  0.2394, -0.3361,  0.8925, -0.4659, -0.3089]])


In [ ]:
import torch
import numpy as np
import os
from tqdm import tqdm

class EmbeddingConverter:
    def __init__(self, model, input_dir, output_dir, batch_size=32, device='cuda'):
        self.model = model.to(device)
        self.model.eval()
        self.input_dir = input_dir
        self.output_dir = output_dir
        self.batch_size = batch_size
        self.device = device
        
        os.makedirs(output_dir, exist_ok=True)
        
    def process_directory(self):
        # Собираем все .npy файлы рекурсивно
        file_list = []
        for root, dirs, files in os.walk(self.input_dir):
            for filename in files:
                if filename.endswith('.npy'):
                    full_path = os.path.join(root, filename)
                    file_list.append(full_path)

        # Обрабатываем файлы
        for input_path in tqdm(file_list, desc="Processing files"):
            # Получаем относительный путь
            rel_path = os.path.relpath(input_path, start=self.input_dir)
            output_path = os.path.join(self.output_dir, rel_path)
            
            # Создаем целевую директорию, если ее нет
            os.makedirs(os.path.dirname(output_path), exist_ok=True)
            
            # Обрабатываем файл
            sequence = np.load(input_path)
            embeddings = self._process_sequence(sequence)
            np.save(output_path, embeddings)
    
    def _process_sequence(self, sequence):
        num_positions = sequence.shape[0]
        embeddings = []
        
        for i in range(0, num_positions, self.batch_size):
            batch = sequence[i:i+self.batch_size]
            batch_tensor = torch.from_numpy(batch).float().to(self.device)
            
            with torch.no_grad():
                batch_embeddings = self.model(batch_tensor).cpu().numpy()
            
            embeddings.append(batch_embeddings)
        
        return np.concatenate(embeddings, axis=0)


# Пример использования:
if __name__ == "__main__":    
    # Создаем конвертер и обрабатываем данные
    converter = EmbeddingConverter(
        model=model,
        input_dir="/home/jupyter/datasphere/project/transformer_dataset/",  # директория с исходными .npy файлами
        output_dir="/home/jupyter/datasphere/project/transformer_dataset_2/",  # директория для сохранения эмбеддингов
        batch_size=64,  # можно настроить в зависимости от доступной памяти
        device='cuda' if torch.cuda.is_available() else 'cpu'
    )
    
    converter.process_directory()